In [23]:
import pandas as pd
import numpy as np
import altair as alt
import streamlit as st
import plotly.express as px

In [24]:
vehicles_df = pd.read_csv('C:/Users/Tyler Wilson/Desktop/Sprint-4-Project/vehicles_us.csv')

In [25]:
# overviewing data
vehicles_df.info()
vehicles_df.sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
17279,10650,2005.0,toyota tundra,excellent,8.0,gas,NaN,automatic,pickup,NaN,1.0,2019-03-26,40
29586,3499,2000.0,ford expedition,excellent,8.0,gas,220100.0,automatic,SUV,NaN,NaN,2018-11-17,20
21383,11995,2012.0,ford explorer,excellent,6.0,gas,121467.0,automatic,wagon,black,1.0,2018-08-23,36
8072,2500,2006.0,honda accord,good,4.0,gas,185482.0,automatic,sedan,silver,NaN,2018-05-05,16
41404,11995,2008.0,honda pilot,like new,6.0,gas,91000.0,automatic,SUV,black,1.0,2019-01-10,7
35692,5995,2003.0,cadillac escalade,excellent,8.0,gas,169518.0,automatic,SUV,red,1.0,2018-05-02,19
43771,14500,2009.0,toyota tacoma,like new,6.0,gas,167000.0,automatic,truck,silver,1.0,2019-03-05,25
19717,13500,2016.0,chrysler town & country,like new,6.0,gas,78000.0,automatic,mini-van,grey,NaN,2019-03-26,6
27117,5600,2010.0,subaru forester,good,4.0,gas,156000.0,automatic,SUV,NaN,NaN,2018-12-08,58
37778,19995,2012.0,ram 1500,excellent,8.0,gas,124839.0,automatic,pickup,white,1.0,2018-07-12,53


In [26]:
# check for duplicate rows
print(vehicles_df.duplicated().sum())

0


In [27]:
# assuming that the NaN values in the 'is_4wd' column mean that the vehicle is not a 4-wheel drive, let's replace them with a 0.
vehicles_df['is_4wd'] = vehicles_df['is_4wd'].fillna(0)

In [29]:
# drop the rest of the rows with missing values
vehicles_df = vehicles_df.dropna().reset_index(drop=True)

In [31]:
# convert float64 columns to int64
vehicles_df['model_year'] = vehicles_df['model_year'].astype('int64')
vehicles_df['cylinders'] = vehicles_df['cylinders'].astype('int64')
vehicles_df['odometer'] = vehicles_df['odometer'].astype('int64')
vehicles_df['is_4wd'] = vehicles_df['is_4wd'].astype('int64')

# convert 'date_posted' column to datetime
vehicles_df['date_posted'] = pd.to_datetime(vehicles_df['date_posted'])


In [53]:
vehicles_df.to_csv('cleaned_vehicles.csv')

Now that the dataframe is cleaned, let's write the code for the web app.

In [50]:
# set title
st.title('Vehicle Data Analysis')

# make scatter plot
st.header('Price vs Odometer Value')

# set checkbox
show_condition_color = st.checkbox('Color by Condition')

if show_condition_color:
    scatter_fig = px.scatter(vehicles_df, x='odometer', y='price', color='condition',
                             title='Price vs Odometer Value (Colored by Condition)',
                             labels={'odometer': 'Odometer (miles)', 'price': 'Price (USD)', 'condition': 'Condition'})
else:
    scatter_fig = px.scatter(vehicles_df, x='odometer', y='price',
                             title='Price vs Odometer Value',
                             labels={'odometer': 'Odometer (miles)', 'price': 'Price (USD)'})

st.plotly_chart(scatter_fig)

# make histogram
st.header('Distribution of Vehicle Conditions')

hist_fig = px.histogram(vehicles_df, x='condition', title='Vehicle Condition Distribution',
                        labels={'condition': 'Condition', 'count': 'Count'})

st.plotly_chart(hist_fig)

2024-07-25 19:44:44.312 
  command:

    streamlit run C:\Users\Tyler Wilson\miniconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()